In [40]:
import os

In [41]:
%pip install langchain-groq langchain-core python-dotenv PyPDF2 pandas

Note: you may need to restart the kernel to use updated packages.


In [42]:
import sys
print(sys.executable)

d:\MCQs_Generator\env\python.exe


In [43]:
! "d:\MCQs_Generator\env\python.exe" -m pip install langchain-groq

In [44]:
! "d:\MCQs_Generator\env\python.exe" -m pip install langchain-community

In [45]:
# from langchain.llms import OpenAI  <-- (Old OpenAI Import)
# from langchain.chat_models import ChatOpenAI <-- (Old OpenAI Import)
from langchain_groq import ChatGroq # <--- (New Groq Import)

# Updated imports for langchain v0.2+
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence

# from langchain.callbacks import get_openai_callback <-- (This specific callback is for OpenAI only. You can remove it or comment it out.)

import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [46]:
load_dotenv()

True

In [47]:
key = os.getenv("GROQ_API_KEY")

In [48]:
llm = ChatGroq(
    groq_api_key = key, # Ensure this is in your .env file
    model_name="llama-3.3-70b-versatile"  # Updated to currently supported Groq model
)

In [49]:
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001BE08318F50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BE08319810>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [50]:
with open("D:\\MCQs_Generator\\Response.json", "r") as f:
    RESPONSE_JSON = json.load(f)

In [51]:
print(RESPONSE_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [52]:
TEMPLATE = """
Text:{text}
You are an expert an MCQ maker. Given the above text, it is your job to create 
a quiz of {number} multiple choice questions for {subject} students in {tone}.
Make sure the questions are not repeated and check all the questions to be confirming.
Make sure to format your response like RESPONSE_JSON below and use it as a guide
Ensure to make {number} MCQs.
### RESPONSE_JSON
{RESPONSE_JSON}
"""

In [53]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
    template=TEMPLATE
    )

In [54]:
# Using modern LCEL (LangChain Expression Language) instead of deprecated LLMChain
quiz_chain = quiz_generation_prompt | llm | StrOutputParser()

In [55]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz
for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at maz 50 words for complexity analysis.
If the quiz is not as per with the cognitive and analytical capabilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [56]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
    )

In [57]:
review_chain = quiz_evaluation_prompt | llm | StrOutputParser()

In [58]:
# Using LCEL to create a combined chain that passes outputs between chains
# RunnablePassthrough allows passing the original inputs along with new outputs
from langchain_core.runnables import RunnableLambda

def create_review_input(inputs):
    """Prepare inputs for review chain by adding quiz output"""
    return {
        **inputs,
        "quiz": inputs.get("quiz", "")
    }

# Combined chain: first generates quiz, then reviews it
generate_evaluate_chain = (
    RunnablePassthrough.assign(quiz=quiz_chain)
    | RunnableLambda(lambda x: {**x, "quiz": x["quiz"]})
    | RunnablePassthrough.assign(review=review_chain)
)

In [59]:
PATH = "D:\\MCQs_Generator\\data.txt"

In [60]:
with open(PATH, "r") as file:
    TEXT = file.read()

In [61]:
print(TEXT)

Generative artificial intelligence, also known as generative AI or GenAI, is a subfield of artificial intelligence that uses generative models to generate text, images, videos, audio, software code or other forms of data.[1] These models learn the underlying patterns and structures of their training data and use them to generate new data[2] in response to input, which often takes the form of natural language prompts.[3][4]

The prevalence of generative AI tools has increased significantly since the AI boom in the 2020s. This boom was made possible by improvements in deep neural networks, particularly large language models (LLMs), which are based on the transformer architecture. Generative AI applications include chatbots such as ChatGPT, Claude, Copilot, DeepSeek, Google Gemini and Grok; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo, LTX and Sora.[5][6][7]

Companies in a variety of sectors have used generative AI, including

In [62]:
#MCQ KO GENERATE KRNA HAI

In [63]:
RESPONSE_JSON

{'1': {'no': '1',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'no': '2',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'no': '3',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [64]:
import json
json.dumps(RESPONSE_JSON)

'{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [65]:
response = generate_evaluate_chain.invoke({
    "text": TEXT,
    "number": 5,
    "subject": "AI",
    "tone": "Simple",
    "RESPONSE_JSON": json.dumps(RESPONSE_JSON)
})
print(response)

{'text': 'Generative artificial intelligence, also known as generative AI or GenAI, is a subfield of artificial intelligence that uses generative models to generate text, images, videos, audio, software code or other forms of data.[1] These models learn the underlying patterns and structures of their training data and use them to generate new data[2] in response to input, which often takes the form of natural language prompts.[3][4]\n\nThe prevalence of generative AI tools has increased significantly since the AI boom in the 2020s. This boom was made possible by improvements in deep neural networks, particularly large language models (LLMs), which are based on the transformer architecture. Generative AI applications include chatbots such as ChatGPT, Claude, Copilot, DeepSeek, Google Gemini and Grok; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo, LTX and Sora.[5][6][7]\n\nCompanies in a variety of sectors have used generative

In [66]:
response

{'text': 'Generative artificial intelligence, also known as generative AI or GenAI, is a subfield of artificial intelligence that uses generative models to generate text, images, videos, audio, software code or other forms of data.[1] These models learn the underlying patterns and structures of their training data and use them to generate new data[2] in response to input, which often takes the form of natural language prompts.[3][4]\n\nThe prevalence of generative AI tools has increased significantly since the AI boom in the 2020s. This boom was made possible by improvements in deep neural networks, particularly large language models (LLMs), which are based on the transformer architecture. Generative AI applications include chatbots such as ChatGPT, Claude, Copilot, DeepSeek, Google Gemini and Grok; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo, LTX and Sora.[5][6][7]\n\nCompanies in a variety of sectors have used generative

In [67]:
quiz = response.get("quiz")

In [68]:
json.loads(quiz)

{'1': {'no': '1',
  'mcq': 'What is Generative Artificial Intelligence (GenAI) used for?',
  'options': {'a': 'To develop operating systems',
   'b': 'To generate text, images, videos, audio, software code or other forms of data',
   'c': 'To create simple chatbots',
   'd': 'To make video games'},
  'correct': 'b'},
 '2': {'no': '2',
  'mcq': 'What architecture is used in large language models (LLMs) for Generative AI?',
  'options': {'a': 'Recurrent Neural Network',
   'b': 'Transformer architecture',
   'c': 'Convolutional Neural Network',
   'd': 'Deep Neural Network'},
  'correct': 'b'},
 '3': {'no': '3',
  'mcq': 'Which of the following is a potential negative impact of Generative AI?',
  'options': {'a': 'Creation of new job opportunities',
   'b': 'Increased environmental sustainability',
   'c': 'Mass replacement of human jobs',
   'd': 'Improved intellectual property laws'},
  'correct': 'c'},
 '4': {'no': '4',
  'mcq': 'Who introduced the concept of the Markov chain in 1906?

In [69]:
quiz = json.loads(quiz)

In [70]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({
        "MCQ": mcq,
        "Choices": options,
        "Correct": correct
    })

In [71]:
quiz_table_data

[{'MCQ': 'What is Generative Artificial Intelligence (GenAI) used for?',
  'Choices': 'a: To develop operating systems | b: To generate text, images, videos, audio, software code or other forms of data | c: To create simple chatbots | d: To make video games',
  'Correct': 'b'},
 {'MCQ': 'What architecture is used in large language models (LLMs) for Generative AI?',
  'Choices': 'a: Recurrent Neural Network | b: Transformer architecture | c: Convolutional Neural Network | d: Deep Neural Network',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is a potential negative impact of Generative AI?',
  'Choices': 'a: Creation of new job opportunities | b: Increased environmental sustainability | c: Mass replacement of human jobs | d: Improved intellectual property laws',
  'Correct': 'c'},
 {'MCQ': 'Who introduced the concept of the Markov chain in 1906?',
  'Choices': 'a: Andrey Markov | b: Harold Cohen | c: Alan Turing | d: Eugeny Onegin',
  'Correct': 'a'},
 {'MCQ': 'What is one of the 

In [72]:
df = pd.DataFrame(quiz_table_data)
df

,MCQ,Choices,Correct
0,What is Generative Artificial Intelligence (Ge...,a: To develop operating systems | b: To genera...,b
1,What architecture is used in large language mo...,a: Recurrent Neural Network | b: Transformer a...,b
2,Which of the following is a potential negative...,a: Creation of new job opportunities | b: Incr...,c
3,Who introduced the concept of the Markov chain...,a: Andrey Markov | b: Harold Cohen | c: Alan T...,a
4,What is one of the environmental impacts of la...,a: High energy consumption | b: Low water cons...,a


In [73]:
df.to_csv("D:\\MCQs_Generator\\quiz.xlsx", index=False)